In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

from pathlib import Path
import pandas as pd
import pickle
import numpy as np
from tqdm.notebook import tqdm

import torch.utils.data
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator, Engine
from ignite.metrics import Accuracy, Loss

from src.models import InsiderClassifier, LSTM_Encoder
from src.params import get_params
from src.dataset import CertDataset, create_data_loaders
from src.trainer import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# TODO:

* Переместить деление на трейн-тест
* Считать точность и лосс только по непаддинговым токенам


In [15]:
# output_dir = Path(r'C:\Users\Mvideo\Google Drive\Datasets\CERT_output')
# answers_dir = Path(r"C:/Users/Mvideo/Downloads/answers")

output_dir = Path(r'C:\Users\admin\Google Drive\Datasets\CERT_output')
answers_dir = Path(r"C:\Users\admin\Google Drive\Datasets\CERT\answers")

run_name = 'final3-nll'

main_answers_file = answers_dir / "insiders.csv"

In [16]:
actions, targets = CertDataset.prepare_dataset(output_dir / 'aggregated.pkl', main_answers_file, min_length=50, max_length=200)

C:\Users\admin\AppData\Roaming\Python\Python36\site-packages\pandas\core\ops\__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


# Train

TODO:
* Эксперименты с LR Scheduling

In [17]:
cert_dataset = CertDataset(actions, targets)
train_loader, val_loader = create_data_loaders(cert_dataset, validation_split=0.3, random_seed=0, batch_size=256)

params = get_params()

device = 'cuda'

In [18]:
lstm_encoder = LSTM_Encoder(params['model']['lstm_encoder'])
criterion = nn.NLLLoss()
optimizer = optim.Adam(lstm_encoder.parameters())

train_engine = create_supervised_trainer_lstm(
                                        lstm_encoder, optimizer, criterion, device=device,
                                        prepare_batch=prepare_batch_lstm,
                                        log_dir=(output_dir / 'logs' / run_name).as_posix(),
                                        checkpoint_dir=output_dir / 'checkpoints' / run_name
                                       )

val_engine = create_supervised_evaluator_lstm(
        lstm_encoder, device=device,
        prepare_batch=prepare_batch_lstm,
        metrics={},
        criterion=criterion,
        log_dir=(output_dir / 'logs' / run_name).as_posix(),
)

@train_engine.on(Events.STARTED)
def log_training_results(trainer):
    print('Starting Validation run:')
    val_engine.run(val_loader)

@train_engine.on(Events.EPOCH_COMPLETED)
def log_training_results(trainer):
    print('Validation run:')
    val_engine.run(val_loader)

# @train_engine.on(Events.EPOCH_COMPLETED)
# def log_validation_results(trainer):
#     val_engine.run(val_batcher)
#     metrics = evaluator.state.metrics
#     print("Validation Results - Epoch: {}  Avg accuracy: {:.2f} Avg loss: {:.2f}"
#           .format(trainer.state.epoch, metrics['accuracy']))


In [ ]:
train_engine.run(train_loader, max_epochs=10)

Starting Validation run:



Validation Results - Epoch: 1  Avg loss: 4.211154, Accuracy: 0.000082



Validation run:



Validation Results - Epoch: 1  Avg loss: 0.269076, Accuracy: 0.942371



Validation run:


...


Validation run:



Validation Results - Epoch: 1  Avg loss: 0.123167, Accuracy: 0.969150



Validation run:



Validation Results - Epoch: 1  Avg loss: 0.119632, Accuracy: 0.969880



Validation run:
